<a href="https://colab.research.google.com/github/Kacper-W-Kozdon/interviewProgram/blob/main/Testing_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
lst = [[1, 2, 3],
       [4, 5, 6, 7],
       [8, 9]]
print(lst[1:3][0:2])


[[4, 5, 6, 7], [8, 9]]


In [ ]:

def wrap(fun):
  def wrapper(*args, **kwargs):
    print(fun.__doc__)
    return fun
  return wrapper

@wrap
def fun():
  '''
  asdfasdf
  '''
fun()






  asdfasdf
  


<function __main__.fun()>

In [ ]:
x = 0
a = list([0])


def test_fun():
  global x
  x += 1

def test_fun2(x):
  x += 1
  return x

def test_fun3(x = x):
  def inside(fun, *args, **kwargs):

    return dict(**kwargs)
  print(inside(test_fun3))
  print()
  print(vars())
  print(globals())
  vars().__setitem__(str(a), list([2]))
  print(vars())
  x[:] = [1, 2, 3]











test_fun()
test_fun2(x)
test_fun3(x = a)
print(x)
print(a)



{}

{'x': [0], 'inside': <function test_fun3.<locals>.inside at 0x7b1e07b51090>}
{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'lst = [[1, 2, 3],\n       [4, 5, 6, 7],\n       [8, 9]]\nprint(lst[1:3][0:2])', "def wrap(fun):\n  def wrapper(*args, **kwargs):\n    print(fun.__doc__)\n    return fun\n  return wrapper\n\n@wrap\ndef fun():\n  '''\n  asdfasdf\n  '''\nfun()", 'x = 0\na = list([0])\n\n\ndef test_fun():\n  global x\n  x += 1\n\ndef test_fun2(x):\n  x += 1\n  return x\n\ndef test_fun3(x = x):\n  def inside(fun, *args, **kwargs):\n\n    return dict(**kwargs)\n  print(inside(test_fun3))\n  print()\n  print(vars())\n  print(globals())\n  vars().__setitem__(str(a), list([2]))\n  print(vars())\n  x[:] = [1, 2, 3]\n\n\n\n\n\n\n\n\n\n\n\ntest_fun()\ntest_fun2(x)\ntes

In [ ]:

test_dict = {"entry1": 0, "entry2": 2}
print(test_dict.get("entry1"))
print(test_dict.get("entry3"))

0
None


In [3]:
import torch
import tensorflow as tf

In [4]:
a = torch.tensor([1, 2, 3, 4])
b = torch.tensor([[5, 6], [7, 8]])
print(a.reshape(b.shape))
print(torch.reshape(a, b.shape))

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]])


In [5]:
def gather_old(
    params: torch.Tensor,
    indices: torch.Tensor,
    /,
    *,
    axis: int = -1,
    batch_dims: int = 0,
    out: torch.Tensor = None,
) -> torch.Tensor:
    axis %= len(params.shape)
    batch_dims %= len(params.shape)
    result = []
    if batch_dims == 0:


      print(params.shape, indices.shape, params.shape < indices.shape, params.dim(), indices.dim())
      dim_diff = params.dim() - indices.dim()
      if dim_diff > 0:
        params_squeezed = params
        for d in range(dim_diff):
          params_squeezed.squeeze(-1)
        result = torch.take(
              params_squeezed, indices.long(), out=None)
        print("res: ", result)

        print("shape: ", (params.shape[:axis] + indices.shape[batch_dims:]
                                + params.shape[axis + 1 :]))
        for d in range(dim_diff):
          result = result.expand((params.shape[:axis] + indices.shape[batch_dims:]
                                + params.shape[axis + 1 :]))

        print("reshaped result: ", result)
      else:
        result = torch.take(
              params, indices.long(), out=None)
        print("else res: ", result)
        print("shape: ", (params.shape[:axis] + indices.shape[batch_dims:]
                                + params.shape[axis + 1 :]))
        result = result.reshape((params.shape[:axis] + indices.shape[batch_dims:]
                                + params.shape[axis + 1 :])
                                )

      print([*params.shape[0:batch_dims], *result.shape[1:]], (params.shape[:axis] + indices.shape[batch_dims:]
                              + params.shape[axis + 1 :]), result, params.shape, indices.shape, params.shape > indices.shape, params.shape == indices.shape
            )

    else:
        params_slices = torch.unbind(params, axis=0)
        indices_slices = torch.unbind(indices, axis=0)
        for b in range(batch_dims):
            if b == 0:
                zip_list = [(p, i) for p, i in zip(params_slices, indices_slices)]
            else:
                zip_list = [
                    (p, i) for z in [zip(p1, i1) for p1, i1 in zip_list] for p, i in z
                ]
        for z in zip_list:
            p, i = z
            print(p.shape, i.reshape((-1, )).shape)
            r = torch.gather(
                p, (axis - batch_dims), torch.reshape(i.long(), (-1, )),
                sparse_grad=False, out=None
            )

            result.append(r)
        result = torch.stack(result)
        #result = torch.cat(result, dim=0)
        print("shape: ", (params.shape[:axis] + indices.shape[batch_dims:]
                                + params.shape[axis + 1 :]))
        result = result.reshape((params.shape[:axis] + indices.shape[batch_dims:]
                                + params.shape[axis + 1 :])
                                )
    return result

In [6]:
def gather(
    params: torch.Tensor,
    indices: torch.Tensor,
    /,
    *,
    axis: int = -1,
    batch_dims: int = 0,
    out: torch.Tensor = None,
) -> torch.Tensor:
    axis %= len(params.shape)
    axis = abs(len(params.shape) + axis) if axis < 0 else axis
    batch_dims %= len(params.shape)
    result = []

    def expand_p_i(params, indices):
        dim_helper_table = [1 for dim in list(params.shape[:axis]
                                              + indices.shape[batch_dims:]
                                              + params.shape[axis + 1:])]
        singleton_dims = torch.Size(dim_helper_table)
        params_ex = params.reshape(params.shape[:axis]
                                   + singleton_dims[batch_dims:]
                                   + params.shape[axis + 1:])
        indices_ex = indices.reshape(singleton_dims[:axis]
                                     + indices.shape[batch_dims:]
                                     + singleton_dims[axis + 1:])
        if (params.shape[:axis]
           + indices.shape[batch_dims:]
           + params.shape[axis + 1:]) != params_ex.shape:
            params_ex = params_ex.expand(params.shape[:axis]
                                         + indices.shape[batch_dims:]
                                         + params.shape[axis + 1:])
        if (indices.shape[:axis]
           + indices.shape[batch_dims:]
           + params.shape[axis + 1:]) != indices_ex.shape:
            indices_ex = indices_ex.expand(params.shape[:axis]
                                           + indices.shape[batch_dims:]
                                           + params.shape[axis + 1:])
        return params_ex, indices_ex
    if batch_dims == 0:
        dim_diff = params.dim() - indices.dim()
        if dim_diff != 0:
            params_expanded, indices_expanded = expand_p_i(params, indices)
            result = torch.gather(
                                    params_expanded, axis,
                                    indices_expanded.long(),
                                    sparse_grad=False, out=out
                                 )
            return result
        else:
            result = torch.gather(
                params, axis, indices.long(), sparse_grad=False, out=out)
            result = result.reshape((params.shape[:axis]
                                    + indices.shape[batch_dims:]
                                    + params.shape[axis + 1:])
                                    )
    else:
        dim_diff = params.dim() - indices.dim()
        params_expanded = params
        indices_expanded = indices
        if dim_diff != 0:
            params_expanded, indices_expanded = expand_p_i(params, indices)
        params_slices = torch.unbind(params_expanded, axis=0)
        indices_slices = torch.unbind(indices_expanded, axis=0)
        for b in range(batch_dims):
            if b == 0:
                zip_list = [(p, i) for
                            p, i in
                            zip(params_slices, indices_slices)
                            ]
            else:
                zip_list = [
                           (p, i) for z in [zip(p1, i1) for p1, i1 in zip_list]
                           for p, i in z
                            ]
        for z in zip_list:
            p, i = z
            r = torch.gather(
                             p, (axis - batch_dims),
                             torch.reshape(i.long(), (-1, )),
                             sparse_grad=False, out=None
                            )

            result.append(r)
        result = torch.stack(result)
        result = result.reshape((params.shape[:axis]
                                 + indices.shape[batch_dims:]
                                 + params.shape[axis + 1:])
                                )

    return result

In [7]:
params = torch.tensor([[-1.],
                 [ 1.],
                 [ 1.]])

indices = torch.tensor([[[0],
                  [0]],

                 [[0],
                  [0]],

                 [[0],
                  [0]]])

paramstf = tf.constant([[-1.],
                 [ 1.],
                 [ 1.]])

indicestf = tf.constant([[[0],
                  [0]],

                 [[0],
                  [0]],

                 [[0],
                  [0]]])

axis = 1
batch_dims = 1

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
torch_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor)
tf_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor(
[[[-1.]
  [-1.]]

 [[ 1.]
  [ 1.]]

 [[ 1.]
  [ 1.]]], shape=(3, 2, 1), dtype=float32)




RuntimeError: ignored

In [ ]:
params = torch.tensor([0])

indices = torch.tensor([0, 0])

paramstf = tf.constant([0])

indicestf = tf.constant([0, 0])

axis = 0
batch_dims = 0

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([0 0], shape=(2,), dtype=int32)


torch.Size([1]) torch.Size([2]) True 1 1
else res:  tensor([0, 0])
shape:  torch.Size([2])
[] torch.Size([2]) tensor([0, 0]) torch.Size([1]) torch.Size([2]) False False
torch:  tensor([0, 0])
torch.Size([1]) torch.Size([2]) True 1 1
else res:  tensor([0, 0])
shape:  torch.Size([2])
[] torch.Size([2]) tensor([0, 0]) torch.Size([1]) torch.Size([2]) False False
torch_to_tf:  tf.Tensor([0 0], shape=(2,), dtype=int32)

tf.Tensor([ True  True], shape=(2,), dtype=bool)


In [ ]:
params = torch.tensor([-1])

indices = torch.tensor([[0]])

paramstf = tf.constant([-1])

indicestf = tf.constant([[0]])

axis = 0
batch_dims = 0

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)


torch.Size([1]) torch.Size([1, 1]) True 1 2
else res:  tensor([[-1]])
shape:  torch.Size([1, 1])
[1] torch.Size([1, 1]) tensor([[-1]]) torch.Size([1]) torch.Size([1, 1]) False False
torch:  tensor([[-1]])
torch.Size([1]) torch.Size([1, 1]) True 1 2
else res:  tensor([[-1]])
shape:  torch.Size([1, 1])
[1] torch.Size([1, 1]) tensor([[-1]]) torch.Size([1]) torch.Size([1, 1]) False False
torch_to_tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)

tf.Tensor([[ True]], shape=(1, 1), dtype=bool)


In [ ]:
params = torch.tensor([[-1]])

indices = torch.tensor([0])

paramstf = tf.constant([[-1]])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)


torch.Size([1, 1]) torch.Size([1]) False 2 1
res:  tensor([-1])
shape:  torch.Size([1, 1])
reshaped result:  tensor([[-1]])
[1] torch.Size([1, 1]) tensor([[-1]]) torch.Size([1, 1]) torch.Size([1]) True False
torch:  tensor([[-1]])
torch.Size([1, 1]) torch.Size([1]) False 2 1
res:  tensor([-1])
shape:  torch.Size([1, 1])
reshaped result:  tensor([[-1]])
[1] torch.Size([1, 1]) tensor([[-1]]) torch.Size([1, 1]) torch.Size([1]) True False
torch_to_tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)

tf.Tensor([[ True]], shape=(1, 1), dtype=bool)


In [ ]:
params = torch.tensor([0])

indices = torch.tensor([0])

paramstf = tf.constant([0])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([0], shape=(1,), dtype=int32)


torch.Size([1]) torch.Size([1]) False 1 1
else res:  tensor([0])
shape:  torch.Size([1])
[] torch.Size([1]) tensor([0]) torch.Size([1]) torch.Size([1]) False True
torch:  tensor([0])
torch.Size([1]) torch.Size([1]) False 1 1
else res:  tensor([0])
shape:  torch.Size([1])
[] torch.Size([1]) tensor([0]) torch.Size([1]) torch.Size([1]) False True
torch_to_tf:  tf.Tensor([0], shape=(1,), dtype=int32)

tf.Tensor([ True], shape=(1,), dtype=bool)


In [ ]:
params = torch.tensor([[0],
                 [0]])

indices = torch.tensor([0])

paramstf = tf.constant([[0],
                 [0]])

indicestf = tf.constant([0])

axis = 1
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor(
[[0]
 [0]], shape=(2, 1), dtype=int32)


torch.Size([2, 1]) torch.Size([1]) False 2 1
res:  tensor([0])
shape:  torch.Size([2, 1])
reshaped result:  tensor([[0],
        [0]])
[1] torch.Size([2, 1]) tensor([[0],
        [0]]) torch.Size([2, 1]) torch.Size([1]) True False
torch:  tensor([[0],
        [0]])
torch.Size([2, 1]) torch.Size([1]) False 2 1
res:  tensor([0])
shape:  torch.Size([2, 1])
reshaped result:  tensor([[0],
        [0]])
[1] torch.Size([2, 1]) tensor([[0],
        [0]]) torch.Size([2, 1]) torch.Size([1]) True False
torch_to_tf:  tf.Tensor(
[[0]
 [0]], shape=(2, 1), dtype=int32)

tf.Tensor(
[[ True]
 [ True]], shape=(2, 1), dtype=bool)


In [ ]:
params = torch.tensor([[-13712,  25013, -11631,  -2072,  -7720, -29788, -27294]])

indices = torch.tensor([3, 4, 1])

paramstf = tf.constant([[-13712,  25013, -11631,  -2072,  -7720, -29788, -27294]])

indicestf = tf.constant([3, 4, 1])

axis = 1
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[-2072 -7720 25013]], shape=(1, 3), dtype=int32)


torch.Size([1, 7]) torch.Size([3]) True 2 1
res:  tensor([-2072, -7720, 25013])
shape:  torch.Size([1, 3])
reshaped result:  tensor([[-2072, -7720, 25013]])
[3] torch.Size([1, 3]) tensor([[-2072, -7720, 25013]]) torch.Size([1, 7]) torch.Size([3]) False False
torch:  tensor([[-2072, -7720, 25013]])
torch.Size([1, 7]) torch.Size([3]) True 2 1
res:  tensor([-2072, -7720, 25013])
shape:  torch.Size([1, 3])
reshaped result:  tensor([[-2072, -7720, 25013]])
[3] torch.Size([1, 3]) tensor([[-2072, -7720, 25013]]) torch.Size([1, 7]) torch.Size([3]) False False
torch_to_tf:  tf.Tensor([[-2072 -7720 25013]], shape=(1, 3), dtype=int32)

tf.Tensor([[ True  True  True]], shape=(1, 3), dtype=bool)


In [ ]:
params = torch.tensor([[-1, -1]])

indices = torch.tensor([0])

paramstf = tf.constant([[-1, -1]])

indicestf = tf.constant([0])

axis = 0
batch_dims = 0



print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[-1 -1]], shape=(1, 2), dtype=int32)


torch.Size([1, 2]) torch.Size([1]) False 2 1
res:  tensor([-1])
shape:  torch.Size([1, 2])
reshaped result:  tensor([[-1, -1]])
[2] torch.Size([1, 2]) tensor([[-1, -1]]) torch.Size([1, 2]) torch.Size([1]) True False
torch:  tensor([[-1, -1]])
torch.Size([1, 2]) torch.Size([1]) False 2 1
res:  tensor([-1])
shape:  torch.Size([1, 2])
reshaped result:  tensor([[-1, -1]])
[2] torch.Size([1, 2]) tensor([[-1, -1]]) torch.Size([1, 2]) torch.Size([1]) True False
torch_to_tf:  tf.Tensor([[-1 -1]], shape=(1, 2), dtype=int32)

tf.Tensor([[ True  True]], shape=(1, 2), dtype=bool)


In [ ]:
params = torch.tensor([[-1]])

indices = torch.tensor([[0]])

paramstf = tf.constant([[-1]])

indicestf = tf.constant([[0]])

axis = 0
batch_dims = 0


print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tf:  tf.Tensor([[[-1]]], shape=(1, 1, 1), dtype=int32)


torch.Size([1, 1]) torch.Size([1, 1]) False 2 2
else res:  tensor([[-1]])
shape:  torch.Size([1, 1, 1])
[1, 1] torch.Size([1, 1, 1]) tensor([[[-1]]]) torch.Size([1, 1]) torch.Size([1, 1]) False True
torch:  tensor([[[-1]]])
torch.Size([1, 1]) torch.Size([1, 1]) False 2 2
else res:  tensor([[-1]])
shape:  torch.Size([1, 1, 1])
[1, 1] torch.Size([1, 1, 1]) tensor([[[-1]]]) torch.Size([1, 1]) torch.Size([1, 1]) False True
torch_to_tf:  tf.Tensor([[[-1]]], shape=(1, 1, 1), dtype=int32)

tf.Tensor([[[ True]]], shape=(1, 1, 1), dtype=bool)


In [ ]:
a = torch.tensor([[1]])
print(a.reshape((1, 1, 1)))

tensor([[[1]]])


In [ ]:
params = torch.tensor([[-1], [-1], [-1]])

indices = torch.tensor([1])

paramstf = tf.constant([[-1], [-1], [-1]])

indicestf = tf.constant([1])

axis = 0
batch_dims = 0
print(indices.unsqueeze(-1).unsqueeze(-1))
print(params.squeeze(-1))
print(params.dim(), indices.dim())
print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

tensor([[[1]]])
tensor([-1, -1, -1])
2 1
tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)


torch.Size([3, 1]) torch.Size([1]) False 2 1
res:  tensor([-1])
shape:  torch.Size([1, 1])
reshaped result:  tensor([[-1]])
[1] torch.Size([1, 1]) tensor([[-1]]) torch.Size([3, 1]) torch.Size([1]) True False
torch:  tensor([[-1]])
torch.Size([3, 1]) torch.Size([1]) False 2 1
res:  tensor([-1])
shape:  torch.Size([1, 1])
reshaped result:  tensor([[-1]])
[1] torch.Size([1, 1]) tensor([[-1]]) torch.Size([3, 1]) torch.Size([1]) True False
torch_to_tf:  tf.Tensor([[-1]], shape=(1, 1), dtype=int32)

tf.Tensor([[ True]], shape=(1, 1), dtype=bool)


In [ ]:
torch.gather(torch.tensor([[-1], [-1], [-1]]), 0, torch.tensor([[1]]))

tensor([[-1]])

In [ ]:
params = torch.tensor([[0], [0], [1]])

indices = torch.tensor([0])

paramstf = tf.constant([[0], [0], [1]])

indicestf = tf.constant([0])

axis = 0
batch_dims = 1

print(params.dim(), indices.dim())

print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

2 1
tf:  tf.Tensor([[0]], shape=(1, 1), dtype=int32)


torch.Size([1]) torch.Size([1])
shape:  torch.Size([1])
torch:  tensor([0])
torch.Size([1]) torch.Size([1])
shape:  torch.Size([1])
torch_to_tf:  tf.Tensor([0], shape=(1,), dtype=int32)

tf.Tensor([[ True]], shape=(1, 1), dtype=bool)


In [ ]:
params = torch.tensor([[0], [1]])

indices = torch.tensor([[0], [0]])

paramstf = tf.constant([[0], [1]])

indicestf = tf.constant([[0]])

axis = 1
batch_dims = 0

print(params.shape)
print(indices.expand((indices.shape[0], params.shape[-1 - axis])))
print(torch.gather(params, axis, indices))
print(params.dim(), indices.dim())
print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()
print(tf_tensor == torch_to_tf)

torch.Size([2, 1])
tensor([[0, 0],
        [0, 0]])
tensor([[0],
        [1]])
2 2
tf:  tf.Tensor(
[[[0]]

 [[1]]], shape=(2, 1, 1), dtype=int32)


torch.Size([2, 1]) torch.Size([2, 1]) False 2 2
else res:  tensor([[0],
        [0]])
shape:  torch.Size([2, 2, 1])


RuntimeError: ignored

In [ ]:
axis = 1
batch_dims = 0

params = torch.tensor([[[0], [2]]])

indices = torch.tensor([[[0], [0]], [[0], [0]]])
print("xxx: ", params.shape[:2] + indices.shape + params.shape[2 + 1:])
#params = params.expand((1, 2, 2))
#params = params.unsqueeze(-1).unsqueeze(-1)
params = params.expand(params.shape[:axis] + indices.shape + params.shape[axis + 1:])
print("params shape: ", params.shape)
indices = indices.unsqueeze(0).unsqueeze(2)
indices = indices.expand(params.shape)

paramstf = tf.constant([[[0], [2]]])

indicestf = tf.constant([[[0], [0]], [[0], [0]]])
print(paramstf.shape, indicestf.shape)

print(torch.gather(params, axis, indices))
print(tf.gather(paramstf, indicestf, axis, batch_dims = batch_dims))

In [ ]:
a = torch.tensor([[1, 2]])
a.shape
a0 = a.expand((3, 3, 2))
a = a0.reshape((3, 3, 1, 2))
a = a.expand((3, 3, 2, 2))
ai0 = torch.ones((3, 2))
ai = ai0.reshape((1, 3, 2, 1))
ai = ai.expand((3, 3, 2, 2))

b = [1 for dim in list(a.shape)]
c = torch.Size(b)
d = torch.ones(c)

restorch = torch.gather(a, 1, ai.long())
torch_to_tf_a = tf.convert_to_tensor(a0, dtype = "int32")
torch_to_tf_ai = tf.convert_to_tensor(ai0, dtype = "int32")
print(torch_to_tf_a.shape, torch_to_tf_ai.shape)
restf = tf.gather(torch_to_tf_a, torch_to_tf_ai, axis = 1, batch_dims = 0)
print(restf)
print()
print(restorch.shape)
restf == tf.convert_to_tensor(restorch, dtype = "int32")


(3, 3, 2) (3, 2)
tf.Tensor(
[[[[1 2]
   [1 2]]

  [[1 2]
   [1 2]]

  [[1 2]
   [1 2]]]


 [[[1 2]
   [1 2]]

  [[1 2]
   [1 2]]

  [[1 2]
   [1 2]]]


 [[[1 2]
   [1 2]]

  [[1 2]
   [1 2]]

  [[1 2]
   [1 2]]]], shape=(3, 3, 2, 2), dtype=int32)

torch.Size([3, 3, 2, 2])


<tf.Tensor: shape=(3, 3, 2, 2), dtype=bool, numpy=
array([[[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]],


       [[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]],


       [[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]]])>

In [14]:
params = torch.tensor([1])

indices = torch.tensor([[0], [0]])

paramstf = tf.constant([1])

indicestf = tf.constant([[0], [0]])

axis = -1
batch_dims = 0

print(params.shape)
print(indices.expand((indices.shape[0], params.shape[-1 - axis])))
print(params.shape[:axis] , indices.shape[batch_dims:] , params.shape[axis + 1:])
print(params.dim(), indices.dim())
print("tf: ", tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims))
print()
print()
print("torch: ", gather(params, indices, axis = axis, batch_dims = batch_dims))
tf_tensor = tf.gather(paramstf, indicestf, axis = axis, batch_dims = batch_dims)
torch_tensor = gather(params, indices, axis = axis, batch_dims = batch_dims)
torch_to_tf = tf.convert_to_tensor(torch_tensor, dtype = "int32")
print("torch_to_tf: ", torch_to_tf)
print()

torch.Size([1])
tensor([[0],
        [0]])
torch.Size([]) torch.Size([2, 1]) torch.Size([1])
1 2
tf:  tf.Tensor(
[[1]
 [1]], shape=(2, 1), dtype=int32)




RuntimeError: ignored